# Imports

In [878]:
import copy
import math
import os
from pprint import pprint
import random
import re
import string
from copy import deepcopy
from functools import cmp_to_key, lru_cache, wraps, reduce
from heapq import nlargest
from itertools import combinations, groupby, islice, permutations, product
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict, deque, namedtuple, Counter
from datetime import datetime

import networkx as nx
from IPython.display import HTML, display

np.set_printoptions(linewidth=500)
display(HTML("<style>.container { width:100% !important; }</style>"))
import z3

In [4]:
def open_source(day):
    with open(f'source/day{day}.txt', 'r') as f:
        lista = f.readlines()
    lista = [x.strip('\n') for x in lista]
    return lista

In [18]:
def create_txt_files():
    files = [f"day{x}" if x > 9 else f"day0{x}" for x in range(1, 26)]
    for file in files:
        try:
            with open(os.path.join(os.getcwd(), "source", file + ".txt"), 'x') as fp:
                pass
            with open(os.path.join(os.getcwd(), "source", file + "_example.txt"), 'x') as fp:
                pass
        except FileExistsError:
            print("file existed but continued loop")
            continue

# Day 1

In [219]:
lista = open_source("01")

In [220]:
pattern = '1|2|3|4|5|6|7|8|9|one|two|three|four|five|six|seven|eight|nine'.split('|')
digits_1 = dict(zip(pattern[:9], pattern[:9]))
digits_2 = dict(zip(pattern, pattern[:9] * 2))

In [225]:
for part in [digits_1, digits_2]:
    res = 0
    for row in lista:
        pos = {}
        for k, v in part.items():
            for m in re.finditer(k, row):
                pos[m.start()] = v
        current = int(pos[min(pos)] + pos[max(pos)])
        res += current
    print(res)

54601
54078


# Day 2

In [235]:
lista = open_source("02")

In [236]:
possible = {"red": 12, "green": 13, "blue": 14}
res_1, res_2 = 0, 0

In [237]:
for row in lista:
    game, setts = row.split(":")
    game = int(game.split()[1])
    colors = {"red": 0, "green": 0, "blue": 0}

    for sett in setts.split(";"):
        for balls in sett.split(","):
            num, c = balls.strip().split()
            if (num := int(num)) > colors[c]:
                colors[c] = num

    if all(colors[c] <= possible[c] for c in colors):
        res_1 += game
    res_2 += math.prod(colors.values())

print(f"{res_1}\n{res_2}")

2776
68638


# Day 3

In [244]:
lista = open_source("03")

In [245]:
numbers, symbols, gears = [], set(), set()
offsets = list(product([1, 0, -1], repeat=2))

In [246]:
for x, row in enumerate(lista):
    for y, cell in enumerate(row):
        if not re.fullmatch(r"\d|\.", cell):
            symbols.add((x, y))
        if "*" == cell:
            gears.add((x, y))
    for m in re.finditer(r'\d+', row):
        numbers.append((int(m.group()), [(x, i) for i in range(m.start(), m.end())]))

In [247]:
for i, part in enumerate([symbols, gears]):
    res_1, res_2 = [], []
    for sym in part:
        matrix = [(sym[0] + d[0], sym[1] + d[1]) for d in offsets]
        matches = {num for num, pos in numbers if any(tup in pos for tup in matrix)}
        res_1.extend(matches)
        if len(matches) == 2:
            res_2.append(math.prod(matches))
    print(sum(res_1) if i == 0 else sum(res_2))

546563
91031374


# Day 4

In [387]:
lista = open_source('04')

In [390]:
res_1 = 0
res_2 = [1] * len(lista)
for i, row in enumerate(lista):
    winning, yours = (set(map(int, part.split())) for part in row.split(":")[1].split('|'))
    score = len(winning & yours)
    res_1 += math.floor(2 ** (score - 1))
    for j in range(i + 1, i + score + 1):
        res_2[j] += res_2[i]
    
print(res_1)
print(sum(res_2))

25174
6420979


# Day 5

In [23]:
lista = open_source('05')

In [24]:
rules = []
for l in lista:
    if "seeds" in l:
        seeds = list(map(int, l.split()[1:]))
    elif "map" in l:
        current_rule = []
    elif l:
        current_rule.append(list(map(int, l.split())))
    elif not l:
        rules.append(current_rule)

def apply_rule(seed, rule):
    for dest, start, size in rule:
        if start <= seed < start + size:
            return seed - start + dest
    return seed

seeds_1 = seeds[:]
for rule in rules:
    seeds_1 = [apply_rule(s, rule) for s in seeds_1]

print(min(seeds_1))

ranges = [(seeds[i],seeds[i]+seeds[i+1]-1) for i in range(0, len(seeds), 2)]
rules = [[[dest-start, start, start+size-1] for dest, start, size in rule] for rule in rules]

def consolidate(span_start, span_end, map_start, map_end):
    if span_end < map_start or map_end < span_start:
        return [], [(span_start, span_end)]
    
    mapped = [(max(span_start, map_start), min(span_end, map_end))]
    extra = [(span_start, map_start - 1), (map_end + 1, span_end)]
    extra = [(a, b) for a, b in extra if a <= b]

    return mapped, extra

def apply_rule_to_range(rule, seed_range):
    unprocessed = [seed_range]
    new_ranges = []

    for offset, map_start, map_end in rule:
        processed_spans = [consolidate(span_start, span_end, map_start, map_end) for span_start, span_end in unprocessed]

        if processed_spans:
            processed, unprocessed = zip(*processed_spans)
            unprocessed = [span for spans in unprocessed for span in spans]
            new_ranges.extend((a + offset, b + offset) for spans in processed for a, b in spans)
        else:
            unprocessed = []

    return new_ranges + unprocessed

for rule in rules:
    ranges = [new_range for r in ranges for new_range in apply_rule_to_range(rule, r)]

print(min(start for start, _ in ranges))

196167384
125742456


# Day 6

In [241]:
lista = open_source('06')

In [240]:
race_1 = dict(zip(map(int, lista[0].split()[1:]), map(int, lista[1].split()[1:])))
race_2_time, race_2_distance = int(re.sub(r'\D', '', lista[0])), int(re.sub(r'\D', '', lista[1]))

def count_wins(total_time, distance):
    return sum(i * (total_time - i) > distance for i in range(total_time))

res_1 = [count_wins(time, distance) for time, distance in race_1.items()]
print(math.prod(res_1))

res_2 = count_wins(race_2_time, race_2_distance)
print(res_2)

1413720
30565288


# Day 7

In [233]:
lista = open_source('07')

In [263]:
cards_1 = ["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "A"]
cards_2 = ["J"] + [card for card in cards_1 if card != "J"]
strength_1 = {v: f"{k:02d}" for k, v in enumerate(cards_1)}
strength_2 = {v: f"{k:02d}" for k, v in enumerate(cards_2)}

def get_hand_score(hand):
    times = 10 ** 10
    J_count = Counter(hand)["J"]
    hand_without_J = hand.replace("J", "")
    counts = Counter(hand_without_J).most_common(2)
    mc1 = counts[0][1] if counts else 0
    mc2 = counts[1][1] if len(counts) > 1 else 0
    total_count = mc1 + J_count
    score_map = {
        5: 7 * times,
        4: 6 * times,
        3: 5 * times if mc2 == 2 else 4 * times,
        2: 3 * times if mc2 == 2 else 2 * times
    }
    return score_map.get(total_count, 1 if J_count == 0 else 2 * times)

def calculate_score(hand, points, part):
    strength = strength_1 if part == 1 else strength_2
    transformed_hand = ''.join(strength[c] for c in hand)
    if part == 1:
        hand = hand.replace("J", "X")
    return int(transformed_hand) + get_hand_score(hand), int(points)

def get_day_7(part):
    hands = sorted([calculate_score(*l.split(), part) for l in lista], key=lambda x: x[0])
    return sum(points * rank for rank, (_, points) in enumerate(hands, 1))

In [264]:
get_day_7(1)

250602641

In [265]:
get_day_7(2)

251037509

# Day 8

In [168]:
lista = open_source('08')

In [220]:
G = nx.MultiDiGraph()
for row in lista[2:]:
    source, targets = row.split(' = ')
    targets = targets.strip('()').split(', ')
    for i, target in enumerate(targets):
        label = 'L' if i == 0 else 'R'
        G.add_edge(source, target, label=label)

In [221]:
def follow_directions(graph, current, end, directions):
    turn_count = 0
    while True:
        for s in directions:
            if current.endswith(end):
                return turn_count
            edges = [(u, v) for u, v, d in graph.edges(current, data=True) if d['label'] == s]
            if edges:
                current = edges[0][1]
            turn_count += 1
        
directions = lista[0]

In [222]:
follow_directions(G, "AAA", "ZZZ", directions)

22357

In [224]:
turn_counts = []
for node in G.nodes:
    if node.endswith("A"):
        turn_counts.append(follow_directions(G, node, "Z", directions))
math.lcm(*turn_counts)

10371555451871

# Day 9

In [303]:
lista = open_source('09')

In [323]:
res_1, res_2 = 0, 0
for l in lista:
    elements = list(map(int, l.split()))
    res_1 += get_next_element(series=elements, last=True)
    res_2 += get_next_element(series=elements, last=False)
print(res_1, res_2)

1684566095 1136


In [322]:
def get_next_element(series, last=True, summ=0, add=True):
    if set(series) == {0}:
        return summ
    if last:
        summ += series[-1]
    else:
        summ += (series[0] if add else -series[0])
        add = not add
    
    return get_next_element([b - a for a, b in zip(series, series[1:])], last, summ, add)

# Day 10

In [914]:
def get_neighbors(matrix, i_in, limit=True):
    neighbors = {}
    for move, (dx, dy) in MOVES.items():
        i_out = (i_in[0] + dx, i_in[1] + dy)
        if 0 <= i_out[0] < matrix.shape[0] and 0 <= i_out[1] < matrix.shape[1]:
            neighbors[move] = i_out
    return limit_to_pipes(matrix, neighbors) if limit else list(neighbors.values())

def limit_to_pipes(matrix, neighbors):
    valid_symbols = {"u": ["F", "|", "7"], "d": ["L", "|", "J"],
                     "l": ["F", "-", "L"], "r": ["J", "-", "7"]}
    return [i for d, i in neighbors.items() if matrix[i] in valid_symbols[d]]

def update_side(matrix, current, move, symbol):
    new_pos = tuple(np.add(current, MOVES[move]))
    if 0 <= new_pos[0] < SHAPE[0] and 0 <= new_pos[1] < SHAPE[1] and matrix[new_pos] == ".":
        matrix[new_pos] = symbol

def mark_sides(matrix, node, last_node):
    side_moves = {'u': ('l', 'r'), 'd': ('r', 'l'), 'l': ('d', 'u'), 'r': ('u', 'd')}
    move_dir = next(k for k, v in MOVES.items() if tuple(np.subtract(last_node, node)) == v)
    left_move, right_move = side_moves[move_dir]
    update_side(matrix, node, left_move, "O")
    update_side(matrix, node, right_move, "I")
    update_side(matrix, last_node, left_move, "O")
    update_side(matrix, last_node, right_move, "I")
            
def bfs_with_path(matrix, start_node):
    visited = set()
    queue = [(start_node, 1, [start_node])]
    longest_path = []
    while queue:
        node, depth, path = queue.pop(0)
        if node not in visited:
            visited.add(node)
            matrix[node] = f"{depth}"
            if depth > len(longest_path):
                longest_path = path.copy()
            for neighbour in get_neighbors(matrix, node):
                if neighbour not in visited:
                    queue.append((neighbour, depth + 1, path + [neighbour]))
    return longest_path

def find_string_indices(matrix, string):
    indices = []
    for i in range(SHAPE[0]):
        for j in range(SHAPE[1]):
            if matrix[i, j] == string:
                indices.append((i, j))
    return indices

In [931]:
print(len(path) / 2)

7004


In [932]:
path

[(103, 118),
 (104, 118),
 (105, 118),
 (106, 118),
 (106, 119),
 (105, 119),
 (104, 119),
 (103, 119),
 (103, 120),
 (103, 121),
 (104, 121),
 (104, 120),
 (105, 120),
 (105, 121),
 (106, 121),
 (106, 120),
 (107, 120),
 (107, 121),
 (108, 121),
 (109, 121),
 (110, 121),
 (110, 120),
 (111, 120),
 (111, 121),
 (111, 122),
 (111, 123),
 (112, 123),
 (112, 124),
 (111, 124),
 (110, 124),
 (110, 123),
 (110, 122),
 (109, 122),
 (109, 123),
 (108, 123),
 (108, 122),
 (107, 122),
 (107, 123),
 (107, 124),
 (108, 124),
 (108, 125),
 (109, 125),
 (110, 125),
 (111, 125),
 (111, 126),
 (110, 126),
 (110, 127),
 (111, 127),
 (111, 128),
 (110, 128),
 (109, 128),
 (109, 127),
 (109, 126),
 (108, 126),
 (108, 127),
 (108, 128),
 (108, 129),
 (108, 130),
 (107, 130),
 (107, 129),
 (106, 129),
 (106, 128),
 (107, 128),
 (107, 127),
 (106, 127),
 (106, 126),
 (107, 126),
 (107, 125),
 (106, 125),
 (106, 124),
 (106, 123),
 (106, 122),
 (105, 122),
 (104, 122),
 (104, 123),
 (105, 123),
 (105, 124),

In [926]:
lista = open_source('10')
PIPES = np.array([list(s) for s in lista], dtype='U10')
MOVES = {"u": (-1, 0), "d": (1, 0), "l": (0, -1), "r": (0, 1)}
SHAPE = PIPES.shape
start = tuple(np.argwhere(PIPES == "S")[0])

first_step, _ = get_neighbors(PIPES, start)
path = bfs_with_path(PIPES, first_step)
ROUTE = np.full(SHAPE, ".", dtype='U10')
ROUTE[start] = "0"

for node in path:
    ROUTE[node] = PIPES[node]
for last_node, node in zip([start] + path, path + [start]):
    mark_sides(ROUTE, node, last_node)

for iy, ix in np.ndindex(ROUTE.shape):
    if ROUTE[iy, ix] not in ["O", "I", "."]:
        ROUTE[iy, ix] = "@"

for oi in ["O", "I"]:
    for _ in range(100):
        indices = find_string_indices(ROUTE, oi)
        for i in indices:
            for n in get_neighbors(ROUTE, i, False):
                if ROUTE[n] == ".":
                    ROUTE[n] = oi

unique, counts = np.unique(ROUTE, return_counts=True)
dict(zip(unique, counts))

{'@': 14010, 'I': 417, 'O': 5173}

In [927]:
for row in ROUTE:
    print(''.join(row))

OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOOOO@@OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOO@@@@OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOO@@@@@@@OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOO@@@@@@@OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOO@@@@OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOOOOOOOOOOOOOOO@@@@@OOOOOOOOOOOOOO@@OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO@@OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
OOOOOOOOOOOOO

In [858]:
with open("source/day10out.txt", 'w') as file:
    for row in ROUTE:
        line = ''.join(row)
        file.write(line + '\n')

# Day 11

In [ ]:
source = open_source('11')

# Day 12

In [ ]:
source = open_source('12')

# Day 13

In [ ]:
source = open_source('13')

# Day 14

# Day 15

In [ ]:
source = open_source('15')
source

# Day 16

# Day 17

# Day 18

# Day 19

In [ ]:
source = open_source('19_example')
source 

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25